### 1. Carregar dados 

### Objetivo
Compreender profundamente o problema e os dados disponíveis fornecidos no Google Drive.

### Tecnologias
- **Python** - Linguagem principal
- **Pandas** - Manipulação de dados
- **NumPy** - Operações numéricas
- **Matplotlib/Seaborn** - Visualizações

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar dados do Google Drive
df_candidatos = pd.read_json('../data/applicants.json', encoding='utf-8').T # T = transpose
df_prospeccoes = pd.read_json('../data/prospects.json', encoding='utf-8').T # T = transpose
df_vagas = pd.read_json('../data/vagas.json', encoding='utf-8').T # T = transpose


### 2.Análise básica

In [14]:
# Análise básica
print("=== ANÁLISE EXPLORATÓRIA ===")
print(f"Candidatos: {df_candidatos.shape}") # 42.482 candidatos com 7 características cada
print(f"Prospecções: {df_prospeccoes.shape}") # 14222 prospecções com 3 características cada
print(f"Vagas: {df_vagas.shape}") # 14081 vagas com 3 características cada


=== ANÁLISE EXPLORATÓRIA ===
Candidatos: (42482, 7)
Prospecções: (14222, 3)
Vagas: (14081, 3)


### 3. Identificar tipos de variáveis

In [15]:
# Identificar tipos de variáveis
print("\n=== TIPOS DE DADOS ===")
print("\n=== Candidatos ===")
print(df_candidatos.dtypes)
print("\n=== Prospecções ===")
print(df_prospeccoes.dtypes)
print("\n=== Vagas ===")
print(df_vagas.dtypes)


=== TIPOS DE DADOS ===

=== Candidatos ===
infos_basicas                object
informacoes_pessoais         object
informacoes_profissionais    object
formacao_e_idiomas           object
cargo_atual                  object
cv_pt                        object
cv_en                        object
dtype: object

=== Prospecções ===
titulo        object
modalidade    object
prospects     object
dtype: object

=== Vagas ===
informacoes_basicas    object
perfil_vaga            object
beneficios             object
dtype: object


### 4. Verificar valores faltantes

In [16]:

# Análise mais detalhada de valores faltantes
def analisar_valores_faltantes(df, nome_dataset):
    print(f"\n=== {nome_dataset.upper()} ===")
    missing = df.isnull().sum()
    total = len(df)
    
    print(f"Total de registros: {total:,}")
    print("\nValores faltantes por coluna:")
    
    for coluna, qtd_faltante in missing.items():
        if qtd_faltante > 0:
            percentual = (qtd_faltante / total) * 100
            print(f"  {coluna:<20}: {qtd_faltante:>6} ({percentual:5.1f}%)")
        else:
            print(f"  {coluna:<20}: {qtd_faltante:>6} (completo ✅)")

# Aplicar para todos os datasets
analisar_valores_faltantes(df_candidatos, "Candidatos")
analisar_valores_faltantes(df_prospeccoes, "Prospecções") 
analisar_valores_faltantes(df_vagas, "Vagas")


=== CANDIDATOS ===
Total de registros: 42,482

Valores faltantes por coluna:
  infos_basicas       :      0 (completo ✅)
  informacoes_pessoais:      0 (completo ✅)
  informacoes_profissionais:      0 (completo ✅)
  formacao_e_idiomas  :      0 (completo ✅)
  cargo_atual         :      0 (completo ✅)
  cv_pt               :      0 (completo ✅)
  cv_en               :      0 (completo ✅)

=== PROSPECÇÕES ===
Total de registros: 14,222

Valores faltantes por coluna:
  titulo              :      0 (completo ✅)
  modalidade          :      0 (completo ✅)
  prospects           :      0 (completo ✅)

=== VAGAS ===
Total de registros: 14,081

Valores faltantes por coluna:
  informacoes_basicas :      0 (completo ✅)
  perfil_vaga         :      0 (completo ✅)
  beneficios          :      0 (completo ✅)


## 5. Estatísticas descritivas



In [17]:
print("\n=== ESTATÍSTICAS DESCRITIVAS ===")
print("\n=== Candidatos ===")
print(df_candidatos.describe())


=== ESTATÍSTICAS DESCRITIVAS ===

=== Candidatos ===
                                            infos_basicas  \
count                                               42482   
unique                                              42482   
top     {'telefone_recado': '', 'telefone': '(11) 9704...   
freq                                                    1   

                                     informacoes_pessoais  \
count                                               42482   
unique                                              42469   
top     {'data_aceite': 'Cadastro anterior ao registro...   
freq                                                    4   

                                informacoes_profissionais  \
count                                               42482   
unique                                              12760   
top     {'titulo_profissional': '', 'area_atuacao': ''...   
freq                                                13570   

                            

# 6. Correlações

In [18]:
# 6. Correlações - CORRIGIDO

print("=== ANÁLISE DE CORRELAÇÕES ===")

# Verificar se existem variáveis numéricas
numeric_cols = df_candidatos.select_dtypes(include=[np.number]).columns
print(f"Colunas numéricas encontradas: {len(numeric_cols)}")
print(f"Colunas: {list(numeric_cols)}")

if len(numeric_cols) == 0:
    print("❌ Não há variáveis numéricas para calcular correlações")
    print("🔧 Tentando extrair dados numéricos dos campos aninhados...")
    
    # Função para extrair dados numéricos dos dicionários
    def extrair_dados_numericos(df):
        dados_numericos = pd.DataFrame()
        
        # Verificar se podemos extrair dados de cada coluna
        for col in df.columns:
            print(f"\nAnalisando coluna: {col}")
            
            # Examinar alguns valores para entender a estrutura
            sample_values = df[col].dropna().head(5)
            print(f"Amostra: {list(sample_values)}")
            
            # Tentar extrair informações numéricas específicas
            if col == 'informacoes_profissionais':
                # Extrair experiência em anos (exemplo)
                try:
                    anos_exp = []
                    for item in df[col]:
                        if pd.notna(item) and item != '':
                            # Aqui você adaptaria baseado na estrutura real
                            anos_exp.append(0)  # Placeholder
                        else:
                            anos_exp.append(np.nan)
                    
                    if any(pd.notna(exp) for exp in anos_exp):
                        dados_numericos[f'{col}_anos_experiencia'] = anos_exp
                except Exception as e:
                    print(f"Erro ao extrair de {col}: {e}")
        
        return dados_numericos
    
    # Tentar extrair dados numéricos
    df_numeric = extrair_dados_numericos(df_candidatos)
    
    if len(df_numeric.columns) > 0:
        print(f"\n✅ Dados numéricos extraídos: {list(df_numeric.columns)}")
        
        # Calcular correlações
        if len(df_numeric.columns) > 1:
            plt.figure(figsize=(10, 8))
            correlacoes = df_numeric.corr()
            sns.heatmap(correlacoes, annot=True, cmap='coolwarm', center=0)
            plt.title('Matriz de Correlação - Dados Extraídos')
            plt.tight_layout()
            plt.show()
        else:
            print("Apenas uma variável numérica - correlação não aplicável")
    else:
        print("❌ Não foi possível extrair dados numéricos")
        print("📊 Criando análise alternativa para dados categóricos...")
        
        # Análise alternativa para dados categóricos
        criar_analise_categorica(df_candidatos)

elif len(numeric_cols) == 1:
    print(f"⚠️ Apenas uma variável numérica: {numeric_cols[0]}")
    print("Correlação não é possível com apenas uma variável")
    
    # Mostrar distribuição da única variável numérica
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    df_candidatos[numeric_cols[0]].hist(bins=30, alpha=0.7, edgecolor='black')
    plt.title(f'Distribuição - {numeric_cols[0]}')
    plt.xlabel(numeric_cols[0])
    plt.ylabel('Frequência')
    
    plt.subplot(1, 2, 2)
    df_candidatos[numeric_cols[0]].plot(kind='box')
    plt.title(f'Box Plot - {numeric_cols[0]}')
    plt.ylabel(numeric_cols[0])
    
    plt.tight_layout()
    plt.show()
    
else:
    print(f"✅ {len(numeric_cols)} variáveis numéricas encontradas")
    
    # Calcular e plotar matriz de correlação
    plt.figure(figsize=(12, 8))
    correlacoes = df_candidatos[numeric_cols].corr()
    sns.heatmap(correlacoes, annot=True, cmap='coolwarm', center=0)
    plt.title('Matriz de Correlação - Variáveis Numéricas')
    plt.tight_layout()
    plt.show()
    
    # Mostrar correlações mais fortes
    print("\nCorrelações mais fortes (|r| > 0.5):")
    for i in range(len(correlacoes.columns)):
        for j in range(i+1, len(correlacoes.columns)):
            corr = correlacoes.iloc[i, j]
            if abs(corr) > 0.5:
                print(f"{correlacoes.columns[i]} ↔ {correlacoes.columns[j]}: {corr:.3f}")

def criar_analise_categorica(df):
    """Cria análise para dados categóricos quando não há correlações numéricas"""
    print("\n=== ANÁLISE CATEGÓRICA ALTERNATIVA ===")
    
    # Análise de completude por campo
    completude = {}
    for col in df.columns:
        valores_nao_vazios = df[col].apply(lambda x: x != '' and pd.notna(x) and x != '{}').sum()
        completude[col] = (valores_nao_vazios / len(df)) * 100
    
    # Plotar completude dos dados
    plt.figure(figsize=(12, 6))
    cols = list(completude.keys())
    values = list(completude.values())
    
    plt.bar(cols, values, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title('Completude dos Dados por Campo (%)')
    plt.ylabel('Percentual de Completude')
    plt.xlabel('Campos')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    
    # Adicionar valores nas barras
    for i, v in enumerate(values):
        plt.text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Resumo
    print(f"\nResumo da completude:")
    for col, perc in completude.items():
        print(f"  {col:<25}: {perc:5.1f}%")

=== ANÁLISE DE CORRELAÇÕES ===
Colunas numéricas encontradas: 0
Colunas: []
❌ Não há variáveis numéricas para calcular correlações
🔧 Tentando extrair dados numéricos dos campos aninhados...

Analisando coluna: infos_basicas
Amostra: [{'telefone_recado': '', 'telefone': '(11) 97048-2708', 'objetivo_profissional': '', 'data_criacao': '10-11-2021 07:29:49', 'inserido_por': 'Luna Correia', 'email': 'carolina_aparecida@gmail.com', 'local': '', 'sabendo_de_nos_por': '', 'data_atualizacao': '10-11-2021 07:29:49', 'codigo_profissional': '31000', 'nome': 'Carolina Aparecida'}, {'telefone_recado': '', 'telefone': '(11) 93723-4396', 'objetivo_profissional': 'Analista Administrativo', 'data_criacao': '10-11-2021 08:56:16', 'inserido_por': 'Laura Pacheco', 'email': 'eduardo_rios@hotmail.com', 'local': 'São Paulo, São Paulo', 'sabendo_de_nos_por': 'Outros', 'data_atualizacao': '11-11-2021 11:10:31', 'codigo_profissional': '31001', 'nome': 'Eduardo Rios'}, {'telefone_recado': '', 'telefone': '(11) 92